In [68]:
# SemanticKITTI - 4D4


import yaml
import numpy as np
from utils import read_ply, obj_color

label_mapping_SemanticKITTI = {
    0: "unlabeled",
    1: "car",
    2: "bicycle",
    3: "motorcycle",
    4: "truck",
    5: "other-vehicle",
    6: "person",
    7: "bicyclist",
    8: "motorcyclist",
    9: "road",
    10: "parking",
    11: "sidewalk",
    12: "other-ground",
    13: "building",
    14: "fence",
    15: "vegetation",
    16: "trunk",
    17: "terrain",
    18: "pole",
    19: "traffic-sign",
}

dir_path = "/nodes/veltins/work/fradlin/Interactive_dataset/interactive4d_data/scene_01_SemanticKITTI_08_000238/"
# dir_path = "/nodes/veltins/work/fradlin/Interactive_dataset/interactive4d_data/scene_02_SemanticKITTI4d4_08_236_239/"

scan_path = dir_path + "scan.ply"
obj2label_path = dir_path + "obj2label.yaml"

point_cloud = read_ply(scan_path)
obj2label_file = yaml.safe_load(open(obj2label_path, "r"))
obj2label = {}
for key, value in obj2label_file.items():
    obj2label[int(key)] = int(value)

gt_masks_colors = np.stack((point_cloud["red"], point_cloud["green"], point_cloud["blue"]), axis=-1)
coordinates = np.stack((point_cloud["x"], point_cloud["y"], point_cloud["z"]), axis=-1)
labels = point_cloud["label"]

# Create an empty dictionary to store label-color associations
label_to_color = {}
# Iterate over the labels and corresponding colors
for label, color in zip(labels, gt_masks_colors):
    color_tuple = tuple(color)  # Convert the color array to a tuple (for hashable key)

    # Check if the label is already in the dictionary
    if label in label_to_color:
        # Ensure consistency: check if the existing color matches the current color
        if label_to_color[label] != color_tuple:
            raise ValueError(f"Inconsistent color found for label {label}. " f"Existing color: {label_to_color[label]}, New color: {color_tuple}")
    else:
        # If the label is not in the dictionary, add it with its associated color
        label_to_color[int(label)] = color_tuple

# Now label_to_color contains the consistent label-color associations
# sort the dictionary by label
label_to_color = dict(sorted(label_to_color.items(), key=lambda item: item[0]))
print("Label to color mapping:")
for label, color in label_to_color.items():
    panoptic_label = obj2label[label]
    semantic_label = panoptic_label & 0xFFFF
    semantic_name = label_mapping_SemanticKITTI[semantic_label]
    print(f"Label: {label}, Color: {color}, Object: {semantic_label} - {semantic_name}")

# Formatting the output as requested
formatted_output = "obj_color = {"
formatted_output += ", ".join(f"{label}: {color}" for label, color in label_to_color.items())
formatted_output += "}"
formatted_output = formatted_output.replace("(", "[").replace(")", "]")
print(formatted_output)

Label to color mapping:
Label: 1, Color: (255, 0, 255), Object: 9 - road
Label: 2, Color: (75, 0, 75), Object: 11 - sidewalk
Label: 3, Color: (0, 200, 255), Object: 13 - building
Label: 4, Color: (0, 175, 0), Object: 15 - vegetation
Label: 5, Color: (0, 60, 135), Object: 16 - trunk
Label: 6, Color: (80, 240, 150), Object: 17 - terrain
Label: 7, Color: (150, 240, 255), Object: 18 - pole
Label: 8, Color: (245, 150, 100), Object: 1 - car
Label: 9, Color: (200, 40, 255), Object: 7 - bicyclist
Label: 10, Color: (245, 150, 100), Object: 1 - car
obj_color = {1: [255, 0, 255], 2: [75, 0, 75], 3: [0, 200, 255], 4: [0, 175, 0], 5: [0, 60, 135], 6: [80, 240, 150], 7: [150, 240, 255], 8: [245, 150, 100], 9: [200, 40, 255], 10: [245, 150, 100]}


In [70]:
import yaml
import os
import numpy as np
from sklearn.cluster import KMeans
from preprocess.ply import read_ply, write_ply
from utils import obj_color


def generate_distinct_colors_kmeans(n):
    # Sample a large number of colors in RGB space
    np.random.seed(0)
    large_sample = np.random.randint(0, 256, (10000, 3))

    # Apply k-means clustering to find n clusters
    kmeans = KMeans(n_clusters=n, n_init=10).fit(large_sample)
    colors = kmeans.cluster_centers_.astype(int)

    return [tuple(color) for color in colors]


def labels_to_colors(labels):
    import numpy as np

    unique_labels = np.unique(labels)
    num_colors = len(unique_labels)
    # Generate distinct colors
    colors_list = generate_distinct_colors_kmeans(num_colors)
    # Create a mapping from labels to colors
    label_to_color = {label: colors_list[i] for i, label in enumerate(unique_labels)}
    # Apply the color map
    colors = np.array([label_to_color[int(label.item())] for label in labels])
    return colors


# dir_path = "/nodes/veltins/work/fradlin/Interactive_dataset/interactive4d_data/scene_02_SemanticKITTI4d4_08_236_239/"
dir_path = "/nodes/veltins/work/fradlin/Interactive_dataset/interactive4d_data/scene_01_SemanticKITTI_08_000238/"
scan_path = dir_path + "scan.ply"

point_cloud = read_ply(scan_path)
coordinates = np.stack((point_cloud["x"], point_cloud["y"], point_cloud["z"]), axis=-1)
features = np.stack((point_cloud["time"], point_cloud["intensity"], point_cloud["distance"]), axis=-1)
labels = point_cloud["label"]
colors = np.stack((point_cloud["red"], point_cloud["green"], point_cloud["blue"]), axis=-1)
for label in np.unique(labels):
    mask = labels == label
    color = obj_color[label]
    colors[mask] = color


# colors = labels_to_colors(labels)[:, :3].astype(np.uint8)

field_names = ["x", "y", "z", "red", "green", "blue", "time", "intensity", "distance", "label"]
ply_path = os.path.join(dir_path, f"scan_updated.ply")
write_ply(ply_path, [coordinates, colors, features, labels], field_names)

True

In [65]:
# drop fence

import yaml
import os
import numpy as np
from preprocess.ply import read_ply, write_ply

# dir_path = "/nodes/veltins/work/fradlin/Interactive_dataset/interactive4d_data/scene_02_SemanticKITTI4d4_08_236_239/"
dir_path = "/nodes/veltins/work/fradlin/Interactive_dataset/interactive4d_data/scene_01_SemanticKITTI_08_000238/"
scan_path = dir_path + "scan.ply"

point_cloud = read_ply(scan_path)
coordinates = np.stack((point_cloud["x"], point_cloud["y"], point_cloud["z"]), axis=-1)
features = np.stack((point_cloud["time"], point_cloud["intensity"], point_cloud["distance"]), axis=-1)
labels = point_cloud["label"]
colors = np.stack((point_cloud["red"], point_cloud["green"], point_cloud["blue"]), axis=-1)

mask = labels != 0
coordinates = coordinates[mask]
features = features[mask]
labels = labels[mask]
colors = colors[mask]


field_names = ["x", "y", "z", "red", "green", "blue", "time", "intensity", "distance", "label"]
ply_path = os.path.join(dir_path, f"scan.ply")
write_ply(ply_path, [coordinates, colors, features, labels], field_names)

True